Для начала загрузим данные и преобразуем их в dataframe.

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np

critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

df = DataFrame(critics)
df.columns.name = 'name'
df.index.name = 'movie'
df

name,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Michael Phillips,Mick LaSalle,Toby
movie,,,,,,,
Just My Luck,3.0,1.5,NaN,3.0,NaN,2,NaN
Lady in the Water,NaN,3.0,3.0,2.5,2.5,3,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,3.0,4,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.5,3,4.0
The Night Listener,4.5,3.0,3.0,3.0,4.0,3,NaN
"You, Me and Dupree",2.5,3.5,3.5,2.5,NaN,2,1.0


Рассчитаем матрицу схожести пользователей использую коэффициент Пирсона. Имея dataframe сделать можно с помощью метода corr(). Я так же сразу преобразую матрицу в таблицу, переименую столбцы и уберу корреляции пользователей с самимими собой.

In [10]:
corr = df.corr().stack() # транспонируем и находим корреляции
corr.index.names = [u'name',u'corr_name']
corr = corr.reset_index() 
corr.rename(columns={0:'corr'},inplace=True)
corr = corr[(corr['name'] != corr['corr_name']) # уберем корреляцию с самим собой
           &(corr['corr']>0)] # уберем отрицательные корреляции
corr.head()

,name,corr_name,corr
1,Claudia Puig,Gene Seymour,0.314970
2,Claudia Puig,Jack Matthews,0.028571
3,Claudia Puig,Lisa Rose,0.566947
4,Claudia Puig,Michael Phillips,1.000000
5,Claudia Puig,Mick LaSalle,0.566947


In [8]:
df1 = df.stack().reset_index()
df1.rename(columns={0:'rating'},inplace=True)
df1.head()

,movie,name,rating
0,Just My Luck,Claudia Puig,3.0
1,Just My Luck,Gene Seymour,1.5
2,Just My Luck,Lisa Rose,3.0
3,Just My Luck,Mick LaSalle,2.0
4,Lady in the Water,Gene Seymour,3.0


In [28]:
ratings_and_corrs = pd.merge(corr,df1,
                    left_on='corr_name', right_on='name',
                    how='left')
ratings_and_corrs.drop(['name_y'],axis=1,inplace=True)
ratings_and_corrs.head()

,name_x,corr_name,corr,movie,rating
0,Claudia Puig,Gene Seymour,0.31497,Just My Luck,1.5
1,Claudia Puig,Gene Seymour,0.31497,Lady in the Water,3.0
2,Claudia Puig,Gene Seymour,0.31497,Snakes on a Plane,3.5
3,Claudia Puig,Gene Seymour,0.31497,Superman Returns,5.0
4,Claudia Puig,Gene Seymour,0.31497,The Night Listener,3.0


In [34]:
ratings_and_corrs.set_index(['name_x','corr_name','corr','movie']).xs('Toby',level=0).unstack()

rating                                      \
movie                  Just My Luck Lady in the Water Snakes on a Plane   
corr_name     corr                                                        
Claudia Puig  0.893405          3.0               NaN               3.5   
Gene Seymour  0.381246          1.5               3.0               3.5   
Jack Matthews 0.662849          NaN               3.0               4.0   
Lisa Rose     0.991241          3.0               2.5               3.5   
Mick LaSalle  0.924473          2.0               3.0               4.0   

                                                                               
movie                  Superman Returns The Night Listener You, Me and Dupree  
corr_name     corr                                                             
Claudia Puig  0.893405              4.0                4.5                2.5  
Gene Seymour  0.381246              5.0                3.0                3.5  
Jack Matthews 0.662849              5.0                3.0                3.5  
Lisa Rose     0.991241              3.5                3.0                2.5  
Mick LaSalle  0.924473              3.0                3.0                2.0

In [58]:
def weighted(df):
    prediction = (df['corr']*df['rating']).sum() / df['corr'].sum()
    return pd.Series([prediction], index=['pred'])
predictions = ratings_and_corrs.groupby(['name_x','movie']).apply(weighted)
predictions.unstack()

pred                                      \
movie            Just My Luck Lady in the Water Snakes on a Plane   
name_x                                                              
Claudia Puig         2.282609          2.683756          3.705042   
Gene Seymour         2.633268          2.848029          3.861740   
Jack Matthews        2.150559          2.785639          3.755088   
Lisa Rose            2.236927          2.905560          3.894482   
Michael Phillips     2.809276          2.728130          3.538659   
Mick LaSalle         2.607295          2.755949          3.880319   
Toby                 2.530981          2.832550          3.705974   

                                                                         
movie            Superman Returns The Night Listener You, Me and Dupree  
name_x                                                                   
Claudia Puig             3.701298           3.403643           1.960090  
Gene Seymour             4.094185           3.295503           2.575387  
Jack Matthews            4.113369           3.085205           2.447916  
Lisa Rose                4.093714           3.463311           2.305536  
Michael Phillips         4.078659           3.860112           2.694637  
Mick LaSalle             4.120362           3.476669           2.218058  
Toby                     3.902420           3.347790           2.651006

In [88]:
mask = df.T.isnull().stack()
predictions[mask.values]

pred
name_x           movie                       
Claudia Puig     Lady in the Water   2.683756
Jack Matthews    Just My Luck        2.150559
Michael Phillips Just My Luck        2.809276
                 You, Me and Dupree  2.694637
Toby             Just My Luck        2.530981
                 Lady in the Water   2.832550
                 The Night Listener  3.347790

In [93]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(5))

# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

ratings = lens[['movie_id','user_id','rating']]
ratings.head()

,movie_id,user_id,rating
0,1,308,4
1,4,308,5
2,5,308,4
3,7,308,4
4,8,308,5


In [94]:
df = ratings.pivot_table(index='user_id',columns='movie_id')
similarity = lambda column1, column2: 1/(1+ sum(pow(column1[(~column1.isnull())&(~column2.isnull())] 
                                                    - column2[(~column1.isnull())&(~column2.isnull())],2)))
result = df.apply(lambda col1: df.apply(lambda col2: similarity(col1, col2)))
result.head()

rating                                              ...             \
movie_id   1    2    3    4    5    6    7    8    9    10   ...  1673 1674   
user_id                                                      ...              
1             5    3    4    3    3    5    4    1    5    3 ...   NaN  NaN   
2             4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN    2 ...   NaN  NaN   
3           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
4           NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   
5             4    3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN ...   NaN  NaN   

                                                  
movie_id 1675 1676 1677 1678 1679 1680 1681 1682  
user_id                                           
1         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
5         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 1682 columns]

In [120]:
result2 = result.stack()
result2.index.names = [None,'movie_id','corr_movie_id']
result2 = result2.reset_index()
result2.drop(['level_0'],axis=1,inplace=True)
result2.to_csv('eucleadian.csv',index=False)
result2.head()

,movie_id,corr_movie_id,rating
0,1,1,1.000000
1,1,2,0.005376
2,1,3,0.005000
3,1,4,0.003788
4,1,5,0.012346


In [139]:
# отберем топ 10 рекомендаций
sorting = lambda g: g.sort_values(by='rating').head(10)
top_10 = result2.groupby(['movie_id']).apply(sorting)
top_10.drop(['movie_id'],axis=1,inplace=True)
top_10 = top_10.reset_index()
top_10.drop(['level_1'],axis=1,inplace=True)
top_10.head()

,movie_id,corr_movie_id,rating
0,1,100,0.001706
1,1,50,0.001776
2,1,121,0.001818
3,1,56,0.001916
4,1,235,0.001927


In [151]:
ratings_and_corrs = pd.merge(top_10,ratings[ratings['user_id'] == 1],
                    left_on='corr_movie_id', right_on='movie_id',
                    how='left')
ratings_and_corrs.drop(['movie_id_y'],axis=1,inplace=True)
def weighted(df):
    prediction = (df['rating_x']*df['rating_y']).sum() / df['rating_x'].sum()
    return pd.Series([prediction], index=['pred'])
predictions = ratings_and_corrs.groupby(['user_id','movie_id_x']).apply(weighted)
predictions.head()

pred
user_id movie_id_x          
1       1           4.218900
        2           4.682547
        3           4.784364
        4           4.714331
        5           4.831550

In [176]:
pred_sort = predictions.reset_index().sort_values(['pred'],ascending=False)
pred_sort.head(10)

,user_id,movie_id_x,pred
302,1,303,5
1622,1,1628,5
298,1,299,5
332,1,333,5
306,1,307,5
310,1,311,5
326,1,327,5
329,1,330,5
328,1,329,5
1652,1,1659,5
